# Asyncio Wait vs Gather

Asyncio's API allows has two functions for scheduling coroutines, [wait](https://docs.python.org/3/library/asyncio-task.html#asyncio.wait) and [gather](https://docs.python.org/3/library/asyncio-task.html#asyncio.gather). The are slightly different in behaviour.

In [1]:
import asyncio
import concurrent
from random import random

loop = asyncio.get_event_loop()

async def coro(i):
    sleep_time = random() * 3
    print("Coro with {}, sleeping for {:.2f} seconds".format(i, sleep_time))
    await asyncio.sleep(sleep_time)
    if i == 5 or i == 7:
        raise Exception("Exception on i = {}".format(i))
    return i

## Wait

In [2]:
async def wait():
    tasks = [coro(i) for i in range(10)]
    completed, pending = await asyncio.wait(tasks)
    print("Completed size: {}, pending size {}".format(len(completed), len(pending)))
    for task in completed:
        print(task.result())
        
loop.run_until_complete(wait())

Coro with 7, sleeping for 1.11 seconds
Coro with 6, sleeping for 1.95 seconds
Coro with 0, sleeping for 2.28 seconds
Coro with 8, sleeping for 0.06 seconds
Coro with 5, sleeping for 1.33 seconds
Coro with 1, sleeping for 1.14 seconds
Coro with 4, sleeping for 1.48 seconds
Coro with 3, sleeping for 2.59 seconds
Coro with 9, sleeping for 2.21 seconds
Coro with 2, sleeping for 1.54 seconds
Completed size: 10, pending size 0
8


Exception: Exception on i = 7

Notice all tasks are executed and exceptions are not raised on the actual call.

`wait` returns a tuple of two sets of [Task](https://docs.python.org/3/library/asyncio-task.html#task) objects, one containing the completed Tasks and another one containing the pending ones. We can call `result` on returning the Tasks objects to obtain the result or re-raise any exceptions generated by the coroutines. 

The contents of the sets returned by `wait` is controlled by the `return_when` keyword argument, allowing for constants from `concurrent.futures` FIRST_COMPLETED, FIRST_EXCEPTION and the default ALL_COMPLETED, which results in the pending set being empty by default, but we can tweak the behaviour for finer control on when and what is returned by `wait`.

In [3]:
async def wait_first_exception():
    tasks = [coro(i) for i in range(10)]
    completed, pending = await asyncio.wait(
        tasks, return_when=concurrent.futures.FIRST_EXCEPTION)
    print("Completed size: {}, pending size {}".format(
        len(completed), len(pending)))
    for task in completed:
        try:
            print(task.result())
        except Exception as e:
            print(e)
    for task in pending:
        task.cancel()

loop.run_until_complete(wait_first_exception())

Coro with 3, sleeping for 1.59 seconds
Coro with 9, sleeping for 0.90 seconds
Coro with 0, sleeping for 2.86 seconds
Coro with 4, sleeping for 1.90 seconds
Coro with 1, sleeping for 1.06 seconds
Coro with 5, sleeping for 1.09 seconds
Coro with 6, sleeping for 1.43 seconds
Coro with 7, sleeping for 0.60 seconds
Coro with 2, sleeping for 0.39 seconds
Coro with 8, sleeping for 1.13 seconds
Completed size: 2, pending size 8
2
Exception on i = 7


In [4]:
async def wait_first_completed():
    tasks = [coro(i) for i in range(10)]
    completed, pending = await asyncio.wait(
        tasks, return_when=concurrent.futures.FIRST_COMPLETED)
    print("Completed size: {}, pending size {}".format(
        len(completed), len(pending)))
    for task in completed:
        try:
            print(task.result())
        except Exception as e:
            print(e)
    for task in pending:
        task.cancel()

loop.run_until_complete(wait_first_completed())

Coro with 6, sleeping for 0.74 seconds
Coro with 5, sleeping for 1.29 seconds
Coro with 1, sleeping for 0.51 seconds
Coro with 8, sleeping for 0.33 seconds
Coro with 7, sleeping for 0.32 seconds
Coro with 4, sleeping for 1.26 seconds
Coro with 9, sleeping for 0.86 seconds
Coro with 2, sleeping for 2.79 seconds
Coro with 3, sleeping for 2.11 seconds
Coro with 0, sleeping for 0.41 seconds
Completed size: 1, pending size 9
Exception on i = 7


Additionally `wait` accepts an additional `timeout` keyword argument to force its return after the value, which does not raise a `TimeoutError` but places the task in the second "pending" set.

## Gather

In [5]:
async def gather(return_exceptions=False):
    tasks = [coro(i) for i in range(10)]
    completed = await asyncio.gather(
        *tasks, return_exceptions=return_exceptions)
    print("Tasks complete")
    print(completed)
    
loop.run_until_complete(gather())

Coro with 8, sleeping for 0.10 seconds
Coro with 3, sleeping for 1.43 seconds
Coro with 1, sleeping for 0.10 seconds
Coro with 9, sleeping for 2.55 seconds
Coro with 4, sleeping for 1.61 seconds
Coro with 2, sleeping for 2.91 seconds
Coro with 5, sleeping for 1.02 seconds
Coro with 6, sleeping for 2.49 seconds
Coro with 7, sleeping for 2.53 seconds
Coro with 0, sleeping for 1.35 seconds


Exception: Exception on i = 5

Gather also schedules all tasks but raises the exception as a result of the call, this means the caller will need wrap `gather` in a try/except clause instead of `task.result()`. It does accept a keyword argument `return_exceptions` to prevent this behaviour.

In [6]:
loop.run_until_complete(gather(return_exceptions=True))

Coro with 5, sleeping for 1.35 seconds
Coro with 8, sleeping for 2.83 seconds
Coro with 1, sleeping for 0.20 seconds
Coro with 6, sleeping for 0.39 seconds
Coro with 9, sleeping for 0.09 seconds
Coro with 0, sleeping for 0.39 seconds
Coro with 2, sleeping for 0.66 seconds
Coro with 7, sleeping for 0.26 seconds
Coro with 3, sleeping for 1.29 seconds
Coro with 4, sleeping for 0.98 seconds
Tasks complete
[0, 1, 2, 3, 4, Exception('Exception on i = 5',), 6, Exception('Exception on i = 7',), 8, 9]


Notice the coroutines are executed out of order but the results are *ordered*, not only that but the actual contents of the list are not Task objects but the actual result of the coroutine, and in this case since we passed `return_exceptions` as True the call did not raise the exceptions but included the instances as the results.

## Conclusions

Seems like `gather` is designed for situations where order is important and it's unlikely coroutines will raise errors and you just want the results without dealing with Task objects.

Beyond that `wait` allows finer control over the coroutines being scheduled with the `return_when` and the `timeout` keyword arguments.